<a href="https://colab.research.google.com/github/sunyingjian/study-tensorflow/blob/master/%E8%87%AA%E5%AE%9A%E4%B9%89%E5%BE%AA%E7%8E%AF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [94]:
#标准开场,引入tf2.0
%tensorflow_version 2.x
import tensorflow as tf
print('tensorflow:',format(tf.__version__))
import numpy as np
from tensorflow import keras
import glob
from tensorflow.keras import layers
import matplotlib.pyplot as plt
%matplotlib inline

tensorflow: 2.2.0-rc1


In [0]:
#引入minist数据集
(train_image,train_label),(test_image,test_label)=tf.keras.datasets.mnist.load_data()

In [96]:
#给图像增加维度
train_image=tf.expand_dims(train_image,-1)
train_image.shape

TensorShape([60000, 28, 28, 1])

In [0]:
#预处理，改变数据类型
train_image=tf.cast(train_image/255,tf.float32)

In [0]:
#预处理，转换label的数据类型
train_label=tf.cast(train_label,tf.int64)

In [0]:
#把它作为tf.data加载进来,建立dataset
dataset=tf.data.Dataset.from_tensor_slices((train_image,train_label))

In [100]:
dataset

<TensorSliceDataset shapes: ((28, 28, 1), ()), types: (tf.float32, tf.int64)>

In [0]:
#对数据进行乱序，这里对10000个数据进行乱序混淆
#批次的大小,用batch
dataset=dataset.shuffle(10000).batch(32)
#这里没有设置repeat

In [0]:
#建立模型，进行自定义训练,把整个网络写在list里
#这里input_shape如何让添加任意形状的图片呢，我们可以这样写参数(none,none,1)
model=tf.keras.Sequential([tf.keras.layers.Conv2D(16,[3,3],activation='relu',input_shape=(28,28,1)),
                           tf.keras.layers.Conv2D(32,[3,3],activation='relu'),
                           tf.keras.layers.GlobalAveragePooling2D(),
                           tf.keras.layers.Dense(10)
                           ])
#这里最后一层我没有写激活函数所以我们再后面的lossfunction里有写from_logitc=ture

In [103]:
#看一下model的可训练参数
model.trainable_variables

[<tf.Variable 'conv2d_10/kernel:0' shape=(3, 3, 1, 16) dtype=float32, numpy=
 array([[[[-0.07686789, -0.18151356,  0.19547184,  0.09897722,
            0.1829211 , -0.0162932 ,  0.08063139,  0.16012971,
            0.079861  ,  0.04548083,  0.15331103,  0.15243022,
            0.15893464, -0.07146898,  0.04480568, -0.03200126]],
 
         [[ 0.1630524 ,  0.07048644, -0.13700703,  0.10884671,
            0.01934944,  0.16019817,  0.06096049, -0.03569192,
           -0.01076652, -0.12196926, -0.1595283 ,  0.02659971,
            0.11077164, -0.12597829, -0.18050559, -0.11465323]],
 
         [[-0.05416937, -0.1229446 , -0.10492905, -0.15062149,
            0.08520369, -0.07702251,  0.11959337,  0.16205435,
           -0.01575708, -0.17112197,  0.02673237,  0.18654917,
            0.16061677,  0.16225098, -0.12430082,  0.15118508]]],
 
 
        [[[-0.16564983, -0.17514515, -0.0958493 , -0.00970392,
           -0.17207336,  0.10758345,  0.01186967, -0.1008484 ,
           -0.11471857, -0

优化器

In [0]:
optimizer=tf.keras.optimizers.Adam()

自定义 损失函数

In [0]:
loss_func=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [0]:
#设计一个生成器
features,labels=next(iter(dataset))

In [107]:
features.shape

TensorShape([32, 28, 28, 1])

In [108]:
predictions=model(features)
predictions.shape

TensorShape([32, 10])

In [109]:
#哪个最大呢？
tf.argmax(predictions,axis=1)

<tf.Tensor: shape=(32,), dtype=int64, numpy=
array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1])>

In [0]:
#开始训练
#定义一个loss函数
def loss(model,x,y):#x是实际的数据，y是实际的label
  y_=model(x)#y_是预测的label
  return loss_func(y,y_)
#这样的话 我们输入一个model和数据还有标签就会返回一个loss损失函数

In [0]:
#一步优化过程
def train_step(model,images,labels):
  with tf.GradientTape() as t:
    #计算损失函数
    loss_step=loss(model,images,labels)
  grads=t.gradient(loss_step,model.trainable_variables)
  #优化grads 使梯度下降最快
  optimizer.apply_gradients(zip(grads,model.trainable_variables))
  #这里就是应用这个梯度对可训练参数进行优化
  #以上就完成了对一个batch数据的优化

In [0]:
def train():#定义多层训练函数
  for epoch in range(10):
    #然后我们就可以对dataset进行迭代
    for (batch,(image,label)) in enumerate(dataset):
      #这里使用了enumerate枚举的方法
      train_step(model,image,label)
      #这样的话就进行了一次训练
    print('Epoch{} is finshed'.format(epoch))

In [117]:
train()

Epoch0 is finshed
Epoch1 is finshed
Epoch2 is finshed
Epoch3 is finshed
Epoch4 is finshed
Epoch5 is finshed
Epoch6 is finshed
Epoch7 is finshed
Epoch8 is finshed
Epoch9 is finshed


#tf.keras.metics 汇总计算模块

In [118]:
#初始化一个对象
m=tf.keras.metrics.Mean('acc')#mean方法就是一个计算均值的方法
#调用他
m(10)

<tf.Tensor: shape=(), dtype=float32, numpy=10.0>

In [120]:
m(155)

<tf.Tensor: shape=(), dtype=float32, numpy=82.5>

所以这里为什么是82.5是因为他计算了155和10的均值

In [121]:
#返回他计算均值的结果
m.result()

<tf.Tensor: shape=(), dtype=float32, numpy=82.5>

In [123]:
m([30,40])
m.result()

<tf.Tensor: shape=(), dtype=float32, numpy=50.833332>

In [0]:
#重置状态
m.reset_states()

In [128]:
m.result().numpy()

0.0